# Lightning spectral assignment matching

In [58]:
import time
import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.spatial import Delaunay


def hugarian(matrix):
    """
    Huangrian method for assignment problem
    Input: a matrix
    Output: an assignment matrix 
    """
    n, m = matrix.shape
    P = np.mat(np.zeros((n, m)))
    row_ind, col_ind = linear_sum_assignment(-matrix)
    P[row_ind, col_ind] = 1
    return P


def get_adjacency_matrix(tri):
    """
    Get ajdacency matrix from scipy triangulation
    Input: scipy.spatial.Delaunay triangulation
    Output: Weighted adjacency matrix,
            Weighted sparse adjacency matrix,
            Unweighted sparse adjacency matrix
    """
    # Important: edges may contain duplicates
    edges = np.concatenate(
        (tri.vertices[:, :2], tri.vertices[:, 1:], tri.vertices[:, ::2]), axis=0
    )
    n = tri.points.shape[0]
    adj = np.zeros((n, n), dtype=np.int64)
    adj[edges[:, 0], edges[:, 1]] = 1
    return np.clip(adj + adj.T, 0, 1)


def creact_adjacency(nPoints, connectionRate=1):
    """
    Creat rondom graphs by random 2D points
    Input: number of points, connection rate of graph
    Output: adjacency matrix
    """
    X = np.random.rand(nPoints, 2)  # Create random points
    aj = np.zeros((nPoints, nPoints))

    for i in range(nPoints):  # Compute distance matrices
        for j in range(i, nPoints):
            if np.random.random() < connectionRate:
                aj[i, j] = np.linalg.norm(X[i] - X[j])
                aj[j, i] = aj[i, j]

    tri = Delaunay(X)
    aj_tri_bin = np.mat(get_adjacency_matrix(tri))  # Unweighted sparse adjacency matrix
    aj_del = np.mat(np.multiply(aj_tri_bin, aj))  # Weighted sparse adjacency matrix
    aj = np.mat(aj)  # Weighted adjacency matrix
    return aj, aj_del, aj_tri_bin


def compute_lead_egenvecr(Mat, eps=1e-4):
    """
    Compute the leading eigenvector of a given matrix
    Input: a matrix
    return: leading eigenvecor: n*1 matrix
    """
    Mat = np.mat(Mat)
    n, _ = Mat.shape
    vec = np.mat(np.ones((n, 1)))

    for i in range(20):
        tmp_vec = Mat * vec
        tmp_vec = tmp_vec / abs(tmp_vec).max()
        difference = abs(vec - tmp_vec).max()
        vec = tmp_vec
        if difference < eps:
            # print(str(i) + " iterations done")
            break
    return vec


def rank_permutation(vec1, vec2):
    """
    Solve the assignment problem defined by np.kron(vec1, vec2.T)
    input: two vector
    return:an assignment matrix P
    """
    n_points1 = np.shape(vec1)[0]
    n_points2 = np.shape(vec2)[0]
    order1 = np.argsort(-vec1, axis=0)
    order2 = np.argsort(-vec2, axis=0)
    P = np.mat(np.zeros((n_points1, n_points2)))
    for i in range(min(n_points1, n_points2)):
        P[order1[i, 0], order2[i, 0]] = 1
    return P


def graphmatch_LiSA(adjacency1, adjacency2):
    """
    Lightning spectral assignment algorithm for graph matching
    input: two adjacency matrice
    return: an assignment matrix P, running time
    """
    start = time.perf_counter()

    eigen_vec_1 = compute_lead_egenvecr(adjacency1)
    eigen_vec_2 = compute_lead_egenvecr(adjacency2)
    P = rank_permutation(eigen_vec_1, eigen_vec_2)

    end = time.perf_counter()
    runtime = end - start
    return P, runtime


def graphmatch_SMKB(adjacency1, adjacency2, erro=0.1):
    """
    Spectral matching algorithm based on KoopmansBeckmann’s QAP
    input: two adjacency matrice
    return: an assignment matrix P, running time
    """
    print("##################### SM-KB ##############################")

    start = time.perf_counter()
    n, _ = adjacency1.shape
    m, _ = adjacency2.shape
    X = np.mat(np.ones((n, m))) / (n * m)

    for i in range(30):
        x = X
        X = adjacency1 * X * adjacency2
        X = X / X.max()
        err = abs(x - X).max()
        # print(i, err)
        if err < erro:
            # print("converge")
            break

    P = hugarian(X)
    end = time.perf_counter()
    runtime = end - start
    return P, runtime


def graphmatch_SM(adjacency1, adjacency2, erro=0.1):
    """
    Spectral matching algorithm for graph matching
    input: two adjacency matrice
    return: an assignment matrix P, running time
    """
    print("##################### SM ##############################")

    start = time.perf_counter()

    W = np.kron(adjacency1, adjacency2)
    x = compute_lead_egenvecr(W)

    n, _ = adjacency1.shape
    m, _ = adjacency2.shape
    soft_P = x.reshape((n, m))
    P = hugarian(soft_P)

    end = time.perf_counter()
    runtime = end - start
    return P, runtime

## Comparasion in small graphs (100 nodes)

In [63]:
nPoints = 100
aj, aj_del, aj_tri_bin = creact_adjacency(nPoints)

In [64]:
P_SM, time_SM = graphmatch_SM(aj, aj)
accuracy_SM = np.sum(np.diag(P_SM)) / nPoints
print("The running time of SM: " + str(np.round(time_SM, 5)) + "sec")
print("The accuracy of SM: " + str(np.round(accuracy_SM, 1)))

##################### SM ##############################
The running time of SM: 1.32044sec
The accuracy of SM: 1.0


In [65]:
P_SMKB, time_SMKB = graphmatch_SMKB(aj, aj)
accuracy_SMKB = np.sum(np.diag(P_SMKB)) / nPoints
print("The running time of SM-KB: " + str(np.round(time_SMKB, 5)) + "sec")
print("The accuracy of SM-KB: " + str(np.round(accuracy_SMKB, 1)))

##################### SM-KB ##############################
The running time of SM-KB: 0.0036sec
The accuracy of SM-KB: 1.0


In [66]:
P_LiSA, time_LiSA = graphmatch_LiSA(aj, aj)
accuracy_LiSA = np.sum(np.diag(P_LiSA)) / nPoints
print("The running time of LiSA: " + str(np.round(time_LiSA, 5)) + "sec")
print("The accuracy of LiSA: " + str(np.round(accuracy_LiSA, 1)))

The running time of LiSA: 0.00193sec
The accuracy of LiSA: 1.0


## Comparasion in large graphs (5000 nodes)

In [71]:
nPoints = 5000
aj, aj_del, aj_tri_bin = creact_adjacency(nPoints)

In [72]:
P_SMKB, time_SMKB = graphmatch_SMKB(aj, aj)
accuracy_SMKB = np.sum(np.diag(P_SMKB)) / nPoints
print("The running time of SM-KB: " + str(np.round(time_SMKB, 5)) + "sec")
print("The accuracy of SM-KB: " + str(np.round(accuracy_SMKB, 1)))

##################### SM-KB ##############################
The running time of SM-KB: 203.46984sec
The accuracy of SM-KB: 1.0


In [73]:
P_LiSA, time_LiSA = graphmatch_LiSA(aj, aj)
accuracy_LiSA = np.sum(np.diag(P_LiSA)) / nPoints
print("The running time of LiSA: " + str(np.round(time_LiSA, 5)) + "sec")
print("The accuracy of LiSA: " + str(np.round(accuracy_LiSA, 1)))

The running time of LiSA: 0.11065sec
The accuracy of LiSA: 1.0
